## Fun Little Side-Tangent on Art Transfer

- Train a model on one image and re-create another image using the model trained ont eh first image
- 2 Cost Functions to make this possible:
    - Content Loss - Higher level features of the training set --> L2 Norm Loss of final Convuloution Layer & the Target Image
    - Style Loss - Product of correlations between activations of lower, internal layers --> Gram Product
    - Take the lower layers based on the style image and the upper layers based on the content image

In [ ]:
from os import listdir
from os.path import isfile, join
import cv2

In [ ]:
# Helper funtion to show an image with cv2
def cv_show_img(title, image, wait=0):
    cv2.namedWindow(title)
    cv2.startWindowThread()
    cv2.imshow(title, image)
    cv2.waitKey(wait)
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

# Helper funtion to show multiple images at the same time
def cv_show_mult_img(titleArr, imageArr, wait=0):
    for i in range(len(titleArr)):
        cv2.namedWindow(titleArr[i])
        cv2.startWindowThread()
        cv2.imshow(titleArr[i], imageArr[i])
    cv2.waitKey(wait)
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [ ]:
# Load pre-trained neural transfer models
model_path = './model_objects/models/'
models = [f for f in listdir(model_path) if isfile(join(model_path, f))]

img = cv2.imread('./images/eiffeltower.jpg')

In [ ]:
# Loop over models and apply each one to our image
for (i, model) in enumerate(models):
    # Print the model being used
    model_name = model[:-3]
    print(f'{i+1} Model = {model_name}')

    # Load image and associated model
    style = cv2.imread(f'./images/art/{model_name}.jpg')
    neuralStyleModel = cv2.dnn.readNetFromTorch(f'{model_path}{model}')

    # resize to a fixed height/width
    h, w = int(img.shape[0]), int(img.shape[1])
    newW = int((640 / h) * w)
    newImg = cv2.resize(img, (newW, 640), interpolation=cv2.INTER_AREA)

    # Create a blob of the image and perform a forward pass through the network
    blob = cv2.dnn.blobFromImage(newImg, 1.0, (newW, 640), (103.939, 116.779, 123.68), swapRB=False, crop=False)
    neuralStyleModel.setInput(blob)
    output = neuralStyleModel.forward()

    # ReShape output tensor - add back mean subtraction and re-order the color channels
    output = output.reshape(3, output.shape[2], output.shape[3])
    output[0] += 103.939
    output[1] += 116.779
    output[2] += 123.68
    output /= 255
    output = output.transpose(1, 2, 0)

    # Display image, style, output
    style_scaled = cv2.resize(style,None,fx=0.5,fy=0.5,interpolation=cv2.INTER_NEAREST)
    cv_show_mult_img(
        ['Original','Style','Neural Transfer'],
        [newImg, style_scaled, output]
    )